In [1]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/treads/Documents/fusion.json"

In [2]:
import requests
from google.cloud import storage
from datetime import date
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
import os
from io import StringIO
import csv
path = "/Users/treads/Documents/apex/"


In [3]:
params = {'sdate': date.today().strftime('%y%m%d')}
r = requests.get('http://192.168.86.50:80/cgi-bin/outlog.xml', params=params)
file = r.text
root = ET.fromstring(file)

outlog = {}
for child in root:
    for node in child:
        if node.tag in outlog:
            outlog[node.tag].append(node.text)
        else:
            outlog[node.tag] = [node.text]
outlogdf = pd.DataFrame(outlog)
otlogpath = '/Users/treads/Documents/apex/outlogdf.csv'
outlogdf.to_csv('/Users/treads/Documents/apex/outlogdf.csv') 

In [4]:
date = date.today().strftime('%y%m%d')
params = {'sdate': date}
r = requests.get('http://192.168.86.50:80/cgi-bin/datalog.xml', params=params)
file = r.text
root = ET.fromstring(file)
datalog = {}
dates = []
for child in root:
    for node in child:
        if node.tag == 'date':
            currentdate = node.text
        elif node.tag == 'probe':
            dates.append(currentdate)
            for endpoint in node:
                if endpoint.tag in datalog:
                    datalog[endpoint.tag].append(endpoint.text)
                else:
                    datalog[endpoint.tag] = [endpoint.text] 
datalogdf = pd.DataFrame(datalog)
datalogdf['date'] = dates
dtlogpath = '/Users/treads/Documents/apex/datalogs.csv'
datalogdf.to_csv('/Users/treads/Documents/apex/datalogs.csv') 

In [5]:
from google.cloud import storage



def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    bucket_name = bucket_name 
    source_file_name = source_file_name 
    destination_blob_name = destination_blob_name

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print(
        "File {} uploaded to {}.".format(
            source_file_name, destination_blob_name
        )
    )

upload_blob("fusion_logs",dtlogpath,"datalogs"+date+".txt")
upload_blob("fusion_logs",otlogpath,"outlogs"+date+".txt")


File /Users/treads/Documents/apex/datalogs.csv uploaded to datalogs210308.txt.
File /Users/treads/Documents/apex/outlogdf.csv uploaded to outlogs210308.txt.
